In [1]:
import pandas as pd

In [2]:
# Paso 1: Leer informe Excel Clientes
f1 = pd.DataFrame(columns=['CEDULA', 'Nombre y apellido'])
#f1 = pd.read_excel("archivos2/BASE SOCIAL A 31 DE DICIEMBRE DE 2023 APORTES Y CREDITOS.xlsx")
#f1 = pd.read_excel("archivos/BASE DE DATOS AÑO 2023 INGRESOS Y EGRESOS.xlsx")
f1.columns = [i.strip() for i in f1.columns] # Removiendo espacios en columnas
f1 = f1.rename(columns={"NIT":"CEDULA", "Cedula":"CEDULA", "NOMBREINTE":"Nombre y apellido", "AGENCIA":"COD AGENCIA"})
f1["CEDULA"] = f1["CEDULA"].astype('int64')  # Convirtiendo cédula a entero
f1

,CEDULA,Nombre y apellido


In [3]:
# Paso 2: Leer informe Excel Transacciones
f2 = pd.read_excel("ARCHIVOS3/TRANSACCIONES HISTORICOS 2023.xlsx")
#f2 = pd.read_excel("archivos/BD VISIONAMOS.xlsx")
#f2 = pd.read_excel("archivos2/TRANSACCIONES VISIONAMOS.xlsx")

In [4]:
f2.columns = [i.strip() for i in f2.columns] # Removiendo espacios en columnas
f2.columns = [col.upper() for col in f2.columns]
f2 = f2.rename(columns={"CEDULA":"DOCUMENTO1", "TOTAL EFECTIVO":"VALOR", "TIPO DE MOVIMIENTO":"OPERACION",
                       "AGENCIA": "COD. AGENCIA", "TIPODEMOVIMIENTO":"OPERACION", "TOTALEFECTIVO": "VALOR"})

f2["OPERACION"] = f2["OPERACION"].apply(lambda x: "CREDITO" if "C" in x else "DEBITO")
if 'DISPOSITIVO' not in f2.columns: f2['DISPOSITIVO']=''
f2 = f2.rename(columns={'FECHA_REGISTRO':'FECHA'})
if "ESTADO" in f2.columns: f2 = f2[f2["ESTADO"]=="APROBADA"]
if 'FECHA_REGISTRO' in f2.columns: 
    f2["FECHA"] = pd.to_datetime(f2['FECHA'], format=formato) 
else:
    f2["FECHA"] = pd.to_datetime(f2['FECHA'])  
    
f2['Mes'] = f2['FECHA'].dt.to_period('M')  # Extrayendo el mes de la fecha
f2["DOCUMENTO1"] = f2["DOCUMENTO1"].astype('int64', errors='ignore') # Convirtiendo documento a entero
f2['CANAL'] = f2['CANAL'] + ' ' + f2['DISPOSITIVO']
condAgencia = 1 if "COD. AGENCIA" in f2.columns else 0
f2

C:\Users\eilin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '17/03/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\eilin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '26/05/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\eilin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '22/06/2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\eilin\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '24/04/2023' in DD/MM/YYYY format. Provide format or specif

,FECHA,DOCUMENTO1,DOCUMENTO,COD. AGENCIA,VALOR,CUENTA,OPERACION,DESCRIPCION,CANAL,PRODUCTO,DISPOSITIVO,Mes
0,2023-03-17,1078469863,01 12019,1,20000,21050510,CREDITO,APERTURA CUENTA AFROAHORRITO,VENTANILLA,AFROAHORRITO,,2023-03
1,2023-03-17,1078469863,01 12019,1,20000,11050501,CREDITO,APERTURA CUENTA AFROAHORRITO,VENTANILLA,AFROAHORRITO,,2023-03
2,2023-05-26,1077468793,01 12186,1,5000,21050510,CREDITO,APERTURA CUENTA AFROAHORRITO,VENTANILLA,AFROAHORRITO,,2023-05
3,2023-05-26,1077468793,01 12186,1,5000,11050501,CREDITO,APERTURA CUENTA AFROAHORRITO,VENTANILLA,AFROAHORRITO,,2023-05
4,2023-06-22,1077483833,01 12234,1,5000,21050510,CREDITO,APERTURA CUENTA AFROAHORRITO,VENTANILLA,AFROAHORRITO,,2023-06
...,...,...,...,...,...,...,...,...,...,...,...,...
61505,2023-08-15,1003888968,01 12352,1,12000,11050501,CREDITO,TRANSACCION POR TAQUILLA,VENTANILLA,AFROAHORRITOS,,2023-08
61506,2023-08-28,35696521,01 12376,2,299088009,11050502,CREDITO,TRANSACCION POR TAQUILLA,VENTANILLA,AFROAHORRITOS,,2023-08
61507,2023-08-28,35696521,01 12376,2,299088009,11050502,CREDITO,TRANSACCION POR TAQUILLA,VENTANILLA,AFROAHORRITOS,,2023-08
61508,2023-10-10,1036673035,01 12475,1,200000,21050505,CREDITO,TRANSACCION POR TAQUILLA,VENTANILLA,AFROAHORRITOS,,2023-10


In [5]:
# Paso 3: Extrayendo datos de clientes
b1 = f1[["CEDULA", "Nombre y apellido"]].copy()
for col in ["Ingresos", "Egresos", "Activos", "Pasivos"]:
    b1[col] = f1[col].values if col in f1.columns else 0
b1["Codigo"] = 0 # Código actividad económica: CERO
b1 = b1.fillna(0)
b1.to_excel("Clientes.xlsx", index=False)
b1

,CEDULA,Nombre y apellido,Ingresos,Egresos,Activos,Pasivos,Codigo


In [6]:
# Paso 4: Extrayendo datos de transacciones
cols = ['DOCUMENTO1', 'Mes', 'OPERACION', 'CANAL', 'VALOR']
if condAgencia: cols=cols[:-1]+["COD. AGENCIA", "VALOR"]
f2 = f2[cols] # Extrayendo columnas

# Agrupando por documento, mes, operacion y canal: (Hallando conteo y suma del valor)
b2 = f2.groupby(cols[:-1]).agg({'count', 'sum'}).reset_index()
b2.columns = b2.columns.droplevel(level=0)

# Renombrando nuevamente las columnas (Por la agrupacion se borraron los nombres)
b2.columns.values[:len(cols)-1] = cols[:-1]
b2 = b2.rename(columns={"sum":"VALOR", "count":"N_VALOR"})

# Creando variable productos
b2["PRODUCTO"] = "Ahorros"
b2["DOCUMENTO1"] = b2["DOCUMENTO1"].astype('int64') # Convirtiendo documento a entero

# Agregando columna jurisdiccion (COD. AGENCIA) a los datos desde clientes
if condAgencia==0:
    b2 = pd.merge(b2, f1[['CEDULA', 'COD. AGENCIA']], left_on='DOCUMENTO1', right_on='CEDULA', how='left')
    b2 = b2.drop("CEDULA", axis=1)
b2 = b2.fillna("*")

# Renombrar columnas debito y creditos
columns = ["Mes", "DOCUMENTO1", "N_VALOR", "VALOR", "PRODUCTO", 'CANAL', "COD. AGENCIA"]
b21 = b2[b2["OPERACION"]=="DEBITO"][columns].rename(columns={"Mes":"Fecha", "DOCUMENTO1":"CEDULA", 
                                                             'CANAL':"CANAL", "N_VALOR":"N_DEBITO",
                                                             "VALOR":"SUMA_DEBITO", "COD. AGENCIA":"JURISDICCION"})

b22 = b2[b2["OPERACION"]=="CREDITO"][columns].rename(columns={"Mes":"Fecha", "DOCUMENTO1":"CEDULA", 
                                                              'CANAL':"CANAL", "N_VALOR":"N_CREDITO",
                                                              "VALOR":"SUMA_CREDITO", "COD. AGENCIA":"JURISDICCION"})


# Concatenando columnas debito, creditos y ordenando
b2 = pd.concat([b21, b22]).fillna(0).sort_values(by=["CEDULA", "Fecha"])
b2 = b2[["Fecha", "CEDULA", "N_DEBITO", "SUMA_DEBITO", "N_CREDITO", 
         "SUMA_CREDITO", "PRODUCTO", "CANAL", "JURISDICCION"]]
b2.to_excel("Transacciones.xlsx", index=False)
b2

,Fecha,CEDULA,N_DEBITO,SUMA_DEBITO,N_CREDITO,SUMA_CREDITO,PRODUCTO,CANAL,JURISDICCION
0,2023-12,1593137,0.0,0.0,6.0,300000.0,Ahorros,VENTANILLA,1
1,2023-03,1593330,0.0,0.0,5.0,402000.0,Ahorros,VENTANILLA,2
2,2023-04,1593330,0.0,0.0,10.0,814000.0,Ahorros,VENTANILLA,2
3,2023-06,1593330,0.0,0.0,6.0,404000.0,Ahorros,VENTANILLA,2
4,2023-07,1593330,0.0,0.0,6.0,400000.0,Ahorros,VENTANILLA,2
...,...,...,...,...,...,...,...,...,...
8273,2023-10,9012724244,0.0,0.0,15.0,34470000.0,Ahorros,VENTANILLA,2
8274,2023-11,9012724244,0.0,0.0,3.0,11490000.0,Ahorros,VENTANILLA,1
8275,2023-11,9012724244,0.0,0.0,5.0,11490000.0,Ahorros,VENTANILLA,2
8276,2023-12,9012724244,0.0,0.0,3.0,11490000.0,Ahorros,VENTANILLA,1
